Baca data

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import bisect
import numpy as np
from time import time
from sklearn.metrics import accuracy_score
import pandas as pd

with open('CensusIncome.names.txt', 'r') as fname:
    sname = fname.read()
names=np.array(sname[sname.find("age"):].split(".\n"))
#print(names)

#Encode Data Workclass
workclass = preprocessing.LabelEncoder()
workclass.fit(np.array(names[1][names[1].find(":")+2:].split(", ")))
#Handle Unknown Data : '?'
le_classes = workclass.classes_.tolist()
bisect.insort_left(le_classes, '?')
workclass.classes_ = le_classes
#print(workclass.classes_)

#Encode Data Education
education = preprocessing.LabelEncoder()
education.fit(np.array(names[3][names[3].find(":")+2:].split(", ")))
le_classes = education.classes_.tolist()
bisect.insort_left(le_classes, '?')
education.classes_ = le_classes
#print(education.classes_)

marital_status = preprocessing.LabelEncoder()
marital_status.fit(np.array(names[5][names[5].find(":")+2:].split(", ")))
le_classes = marital_status.classes_.tolist()
bisect.insort_left(le_classes, '?')
marital_status.classes_ = le_classes
#print(marital_status.classes_)

occupation = preprocessing.LabelEncoder()
occupation.fit(np.array(names[6][names[6].find(":")+2:].split(", ")))
le_classes = occupation.classes_.tolist()
bisect.insort_left(le_classes, '?')
occupation.classes_ = le_classes
#print(occupation.classes_)

relationship = preprocessing.LabelEncoder()
relationship.fit(np.array(names[7][names[7].find(":")+2:].split(", ")))
le_classes = relationship.classes_.tolist()
bisect.insort_left(le_classes, '?')
relationship.classes_ = le_classes
#print(relationship.classes_)

race = preprocessing.LabelEncoder()
race.fit(np.array(names[8][names[8].find(":")+2:].split(", ")))
le_classes = race.classes_.tolist()
bisect.insort_left(le_classes, '?')
race.classes_ = le_classes
#print(race.classes_)

sex = preprocessing.LabelEncoder()
sex.fit(np.array(names[9][names[9].find(":")+2:].split(", ")))
le_classes = sex.classes_.tolist()
bisect.insort_left(le_classes, '?')
sex.classes_ = le_classes
#print(sex.classes_)

native_country = preprocessing.LabelEncoder()
native_country.fit(np.array(names[13][names[13].find(":")+2:].split(", ")))
le_classes = native_country.classes_.tolist()
bisect.insort_left(le_classes, '?')
native_country.classes_ = le_classes
#print(native_country.classes_)

#Open Data Cencus
with open('CencusIncome.data.txt', 'r') as fdata:
    s = fdata.read()

raw = s.split("\n")
A = []
print(raw[32560])
length = 32560
for i in range(length):
    A.append(raw[i].split(", "))
    
#Mapping from raw data to  
data_length = 14
cencus_data = []
cencus_target = []
for i in range(length):
    temp = []
    for j in range(data_length):
        if j == 7:
            temp.append(relationship.transform([A[i][j]])[0])
        elif j == 1:
            temp.append(workclass.transform([A[i][j]])[0])
        elif j == 3:
            temp.append(education.transform([A[i][j]])[0])
        elif j == 5:
            temp.append(marital_status.transform([A[i][j]])[0])
        elif j == 6:
            temp.append(occupation.transform([A[i][j]])[0])
        elif j == 8:
            temp.append(race.transform([A[i][j]])[0])
        elif j == 9:
            temp.append(sex.transform([A[i][j]])[0]) 
        elif j == 13:
            temp.append(native_country.transform([A[i][j]])[0])
        else:
            temp.append(int(A[i][j]))
    cencus_data.append(temp)
    cencus_target.append(A[i][data_length])

#Encode Target
target = preprocessing.LabelEncoder()
target.fit(cencus_target)

#Ready to Use Data
y = target.transform(cencus_target)    
X = np.array(cencus_data)


52, Self-emp-inc, 287927, HS-grad, 9, Married-civ-spouse, Exec-managerial, Wife, White, Female, 15024, 0, 40, United-States, >50K


In [4]:
kf = KFold(n_splits=10, shuffle=True)

table = []
training_times = []
prediction_times = []
scores = []
i = 0
for train, test in kf.split(X, y):
    x_train = [X[i] for i in train]
    y_train = [y[i] for i in train]
    x_test = [X[i] for i in test]
    y_test = [y[i] for i in test]
    
    t0 = time()
    model = KNeighborsClassifier(n_neighbors=41).fit(x_train, y_train)
    training_time = '{:.6}'.format(time() - t0)
    
    t0 = time()
    pred = model.predict(x_test)
    prediction_time = '{:.6}'.format(time() - t0)
    
    score = accuracy_score(pred, y_test)
    cm = confusion_matrix(y_test, pred)
    i = i+1
    print ("Confusion Matrix Iterasi", i)
    display(pd.DataFrame(confusion_matrix(y_test, pred), columns=['>50K','<=50K'], index=['>50K','<=50K']))
    
    table += [[training_time, prediction_time, score]]
    training_times += [training_time]
    prediction_times += [prediction_time]
    scores += [score]

display(pd.DataFrame(table, columns=['training time', 'prediction time', 'score'], index=range(1,len(table)+1)))

Confusion Matrix Iterasi 1


,>50K,<=50K
>50K,2477,19
<=50K,612,148


Confusion Matrix Iterasi 2


,>50K,<=50K
>50K,2417,12
<=50K,666,161


Confusion Matrix Iterasi 3


...

Confusion Matrix Iterasi 9


,>50K,<=50K
>50K,2439,13
<=50K,649,155


Confusion Matrix Iterasi 10


,>50K,<=50K
>50K,2442,9
<=50K,670,135


,training time,prediction time,score
1,0.201034,0.0872316,0.806204
2,0.198506,0.111809,0.791769
3,0.213716,0.0842261,0.785934
4,0.195019,0.0812471,0.794840
5,0.224121,0.101739,0.800676
6,0.215073,0.112298,0.800676
7,0.209026,0.106813,0.803133
8,0.214067,0.0857296,0.815418
9,0.189003,0.085758,0.796683
10,0.233619,0.0987935,0.791462
